# Getting started

Once you've chosen your scenario, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [55]:
% matplotlib inline
import datetime
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

## Load the data into a DataFrame
# pd.read_csv()
iowa = pd.read_csv('/Users/Tamara/Desktop/Project 3/Iowa_Liquor_sales_sample_10pct.csv')
iowa.head(5)
## Transform the dates if needed, e.g.
# df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%y")

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0,5.55


In [56]:
iowa["Date"] = pd.to_datetime(iowa["Date"])

In [57]:
iowa['State Bottle Cost'] = [dollars.strip('$') for dollars in iowa['State Bottle Cost']]
iowa['State Bottle Retail'] = [dollars.strip('$') for dollars in iowa['State Bottle Retail']]
iowa['Sale (Dollars)'] = [dollars.strip('$') for dollars in iowa['Sale (Dollars)']]

In [58]:
iowa['Zip Code'] = pd.to_numeric(iowa['Zip Code'], errors='coerce')
iowa['County Number'] = pd.to_numeric(iowa['County Number'], errors='coerce')
iowa['Store Number'] = pd.to_numeric(iowa['Store Number'], errors='coerce')
iowa['State Bottle Cost'] = pd.to_numeric(iowa['State Bottle Cost'], errors='coerce')
iowa['State Bottle Retail'] = pd.to_numeric(iowa['State Bottle Retail'], errors='coerce')
iowa['Sale (Dollars)'] = pd.to_numeric(iowa['Sale (Dollars)'], errors='coerce')

In [59]:
iowa.dtypes

Date                     datetime64[ns]
Store Number                      int64
City                             object
Zip Code                        float64
County Number                   float64
County                           object
Category                        float64
Category Name                    object
Vendor Number                     int64
Item Number                       int64
Item Description                 object
Bottle Volume (ml)                int64
State Bottle Cost               float64
State Bottle Retail             float64
Bottles Sold                      int64
Sale (Dollars)                  float64
Volume Sold (Liters)            float64
Volume Sold (Gallons)           float64
dtype: object

In [60]:
iowa.sort_values(by=["Bottle Volume (ml)"], inplace=True)

In [61]:
iowa['Price per Liter'] = iowa['State Bottle Retail']/(iowa['Bottle Volume (ml)']/1000)
iowa['Margin'] = iowa['State Bottle Retail'] - iowa['State Bottle Cost']
iowa.head(5)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Price per Liter,Margin
256074,2015-02-18,4669,DES MOINES,50312.0,77.0,Polk,1012200.0,SCOTCH WHISKIES,885,900754,Two Casks- Caol IIa & Speyburn Sherry,50,5.77,8.66,6,51.96,0.3,0.08,173.2,2.89
90202,2015-11-09,3952,BETTENDORF,52722.0,82.0,Scott,1042100.0,IMPORTED DRY GINS,35,902282,Bombay Sapphire Gin Minis,50,16.00,24.00,20,480.00,1.0,0.26,480.0,8.00
132626,2015-06-24,4669,DES MOINES,50312.0,77.0,Polk,1012200.0,SCOTCH WHISKIES,885,900754,Two Casks- Caol IIa & Speyburn Sherry,50,5.77,8.66,6,51.96,0.3,0.08,173.2,2.89
245016,2015-10-12,4669,DES MOINES,50312.0,77.0,Polk,1081600.0,WHISKEY LIQUEUR,885,901225,Whiskey Liqueur,50,5.25,7.88,6,47.28,0.3,0.08,157.6,2.63
196687,2015-01-28,4669,DES MOINES,50312.0,77.0,Polk,1082900.0,MISC. IMPORTED CORDIALS & LIQUEURS,885,987882,Black Currant Liqueur Creme de Cassis,50,4.72,7.08,6,42.48,0.3,0.08,141.6,2.36


In [62]:
# Finding 2015 Values

iowa.sort_values(by=["Store Number", "Date"], inplace=True)
first_date = pd.Timestamp("20150101")
last_date = pd.Timestamp("20151231")
start_2015 = (iowa['Date'] >= first_date) 
end_2015 = (iowa['Date'] <= last_date)
iowa_sales_2015 = iowa[(start_2015) & (end_2015)]

In [63]:
iowa_sales_2015.sort_values('Date').head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Price per Liter,Margin
126444,2015-01-05,2549,INDIANOLA,50125.0,91.0,Warren,1081030.0,COFFEE LIQUEURS,370,67522,Kahlua Coffee Liqueur Mini,500,6.60,9.90,1,9.90,0.50,0.13,19.80,3.30
77777,2015-01-05,4898,BURLINGTON,52601.0,29.0,Des Moines,1022100.0,TEQUILA,395,89191,Jose Cuervo Especial Reposado Tequila Mini,500,11.50,17.25,1,17.25,0.50,0.13,34.50,5.75
216151,2015-01-05,2618,CEDAR RAPIDS,52402.0,57.0,Linn,1062310.0,SPICED RUM,260,43331,Captain Morgan Spiced Rum Mini,500,5.50,8.25,2,16.50,1.00,0.26,16.50,2.75
70788,2015-01-05,2618,CEDAR RAPIDS,52402.0,57.0,Linn,1011200.0,STRAIGHT BOURBON WHISKIES,65,19061,Jim Beam Mini,500,6.83,10.25,1,10.25,0.50,0.13,20.50,3.42
262558,2015-01-05,2618,CEDAR RAPIDS,52402.0,57.0,Linn,1011200.0,STRAIGHT BOURBON WHISKIES,65,19064,Jim Beam,375,4.76,7.14,6,42.84,2.25,0.59,19.04,2.38


In [64]:
iowa_sales_2015.dropna()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Price per Liter,Margin
193161,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1012100.0,CANADIAN WHISKIES,260,11296,Crown Royal,750,15.00,22.50,12,270.00,9.0,2.38,30.000000,7.50
65214,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1062310.0,SPICED RUM,259,43026,Admiral Nelson Spiced Rum,750,5.78,8.67,12,104.04,9.0,2.38,11.560000,2.89
66346,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1081200.0,CREAM LIQUEURS,305,73055,Rumchata,750,12.50,18.75,12,225.00,9.0,2.38,25.000000,6.25
47659,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1022100.0,TEQUILA,410,88296,Patron Tequila Silver,750,27.00,40.50,12,486.00,9.0,2.38,54.000000,13.50
110707,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1081900.0,MISC. AMERICAN CORDIALS & LIQUEURS,322,75214,Kinky Blue,750,10.00,15.00,12,180.00,9.0,2.38,20.000000,5.00
105659,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1032080.0,IMPORTED VODKA,434,34546,Pearl Vodka,750,7.86,11.79,12,141.48,9.0,2.38,15.720000,3.93
223428,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1041100.0,AMERICAN DRY GINS,55,29287,Barton Gin,1000,3.92,5.88,12,70.56,12.0,3.17,5.880000,1.96
144321,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1012100.0,CANADIAN WHISKIES,260,11297,Crown Royal Canadian Whisky,1000,18.50,27.75,36,999.00,36.0,9.51,27.750000,9.25
22111,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,35,43127,Bacardi Superior Rum,1000,9.50,14.25,36,513.00,36.0,9.51,14.250000,4.75
238277,2015-01-08,2106,CEDAR FALLS,50613.0,7.0,Black Hawk,1031200.0,VODKA FLAVORED,380,41694,Uv Blue (raspberry) Vodka,1000,7.50,11.25,12,135.00,12.0,3.17,11.250000,3.75


In [65]:
by_store = iowa_sales_2015.groupby('Store Number')

In [66]:
by_store['Sale (Dollars)','Volume Sold (Liters)','Margin','Price per Liter', 'Bottles Sold'].agg((np.mean, np.sum))

Sale (Dollars)            Volume Sold (Liters)            \
                       mean        sum                 mean       sum   
Store Number                                                            
2106             277.658861  146326.22            18.466509   9731.85   
2113              63.334830    9310.22             4.488776    659.85   
2130             285.386301  111871.43            17.580026   6891.37   
2152              54.759433    7721.08             4.491986    633.37   
2178             102.633671   24324.18             8.089114   1917.12   
2190              92.539209  121689.06             4.807734   6322.17   
2191             209.888406  125093.49            13.512282   8053.32   
2200              56.604342   22811.55             4.509280   1817.24   
2205              85.699271   24681.39             5.405937   1556.91   
2228              72.758625   17462.07             5.698542   1367.65   
2233             122.627967   29553.34            10.293320   2480.69   
2238             192.828852   11762.56            11.569672    705.75   
2248             147.134913   67682.06             6.120326   2815.35   
2285             251.395784   76927.11            12.615523   3860.35   
2290              72.084732   56514.43             4.625255   3626.20   
2327              54.207090   10245.14             4.350688    822.28   
2353             148.761846   54000.55            12.409504   4504.65   
2367             111.351778    5010.83             7.705556    346.75   
2413             187.245364   90065.02            13.017235   6261.29   
2445              42.043743    7189.48             3.343801    571.79   
2448              43.164720   17740.70             3.030487   1245.53   
2459              68.077130    7352.33             5.622593    607.24   
2460             101.296577   33731.76             8.355736   2782.46   
2465              66.065985   25831.80             4.381586   1713.20   
2475             144.338784   10681.07            14.120811   1044.94   
2478             230.705233   19840.65            12.069767   1038.00   
2487              70.662494   28900.96             6.229462   2547.85   
2498              38.959828    2259.67             3.967586    230.12   
2500             126.481450  149121.63             8.514970  10039.15   
2501             113.062966  144833.66             7.774707   9959.40   
...                     ...        ...                  ...       ...   
5180              62.206667    2239.44             4.868056    175.25   
5181              87.549091     963.04             5.936364     65.30   
5182             145.163333    2612.94             9.150000    164.70   
5183             152.800000     916.80            10.300000     61.80   
5185              82.876154    1077.39             5.615385     73.00   
5186              42.694286     298.86             2.355714     16.49   
5187              51.073614    4239.11             2.630241    218.31   
5188             111.042941    1887.73             5.670588     96.40   
5189             151.050000     604.20             9.750000     39.00   
5190              77.448667    1161.73             4.650000     69.75   
5191              75.975000     303.90             4.875000     19.50   
5192              90.000000      90.00             4.500000      4.50   
5193              46.500000     139.50             3.000000      9.00   
5196              42.678125     682.85             2.373125     37.97   
5198              62.266563    3985.06             3.131719    200.43   
5199             127.888750    2046.22             9.300000    148.80   
5201              96.593333     289.78             2.876667      8.63   
5203              92.849000     928.49             7.950000     79.50   
5204              37.640000     790.44             2.719048     57.10   
5205              19.510000      39.02             1.750000      3.50   
5206              62.612500     500.90             3.293750     26.35  

In [74]:
iowa.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20151231")
mask = (iowa['Date'] >= start_date) & (iowa['Date'] <= end_date)
iowa_sales_2015 = iowa[mask]

In [75]:
# Group by store name
iowa_sales_2015 = iowa_sales_2015.groupby(by=["Store Number"], as_index=False)

In [77]:
# Compute sums, means
iowa_sales_2015 = iowa_sales_2015.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0], # just extract once, should be the same
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})

In [79]:
iowa_sales_2015.columns = [' '.join(col).strip() for col in iowa_sales_2015.columns.values]

In [81]:
# Rename columns
iowa_sales_2015.columns = ['store_number','city','2015_sales_sum','2015_sales_mean','county_number','2015_price_per_liter_mean'
                     ,'zip_code','2015_volume_sold_liters_sum','2015_volume_sold_liters_mean','2015_margin_mean']

In [83]:
iowa_sales_2015.head()

,store_number,city,2015_sales_sum,2015_sales_mean,county_number,2015_price_per_liter_mean,zip_code,2015_volume_sold_liters_sum,2015_volume_sold_liters_mean,2015_margin_mean
0,2106,CEDAR FALLS,146326.22,277.658861,7.0,17.856601,50613.0,9731.85,18.466509,5.166319
1,2113,GOWRIE,9310.22,63.334830,94.0,18.504292,50543.0,659.85,4.488776,5.445102
2,2130,WATERLOO,111871.43,285.386301,7.0,16.835669,50703.0,6891.37,17.580026,4.925842
3,2152,ROCKWELL,7721.08,54.759433,17.0,13.020983,50469.0,633.37,4.491986,4.322624
4,2178,WAUKON,24324.18,102.633671,3.0,16.062136,52172.0,1917.12,8.089114,4.868861


# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

## Record your findings

Be sure to write out anything observations from your exploratory analysis.

# Mine the data
Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to
compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations [here](http://pandas.pydata.org/pandas-docs/stable/groupby.html) for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.

# Refine the data
Look for any statistical relationships, correlations, or other relevant properties of the dataset.

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [ ]:
from sklearn import linear_model


## Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.